# imports and loading DataFrame

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
import pandas as pd
import numpy as np
import datetime


# fact table
sessions_df = pd.read_json("data/sessions.jsonl", lines=True)

# dimension tables
deliveries_df = pd.read_json("data/deliveries.jsonl", lines=True)
products_df = pd.read_json("data/products.jsonl", lines=True)
users_df = pd.read_json("data/users.jsonl", lines=True)

# constant values

In [2]:
MAKE_PLOTS = True
MAKE_PAIRPLOT = True
DATE_FORMAT = "%Y-%m-%dT%H:%M:%S"
PRICE_TRESHOLD = 100_000    # for outliers
WEIGHT_TRESHOLD = 50        # for outliers
NUM_OF_HOURS = 12
SEED = 42
SHOW_ALL_WARNINGS = False
SHOW_ONLY_ONE_WARNING = False

In [3]:
import warnings

if SHOW_ONLY_ONE_WARNING:
    warnings.filterwarnings(action='once')
elif not SHOW_ALL_WARNINGS:
    warnings.filterwarnings('ignore')

# merging all data into one DataFrame and other transformations

## adding a column with time difference in deliveries
adding a column with time difference between purchase_timestamp and delivery_timestamp in deliveries table

1. Cut microseconds from delivery_timestamp, so it will be the same format as purchase_timestamp, because there are no microseconds in purchase_timestamp (using "." as a separator).
2. Change columns format to datetime
3. Add time_diff column (as timedelta64 object).
4. Drop rows where time_diff is null (which means that delivery_timestamp was null).
5. Change type of time_diff from timedelta64 to seconds in float.
6. Drop rows where time_diff is below 0. THIS STEP IS MADE IN ### without time_diff below 0

In [4]:
# 1.
deliveries_df["delivery_timestamp"] = deliveries_df["delivery_timestamp"].str.split('.', expand=True)[0]

# 2.
deliveries_df["purchase_timestamp"] = pd.to_datetime(deliveries_df["purchase_timestamp"], format=DATE_FORMAT)
deliveries_df["delivery_timestamp"] = pd.to_datetime(deliveries_df["delivery_timestamp"], format=DATE_FORMAT)

# 3.
deliveries_df["time_diff"] = deliveries_df["delivery_timestamp"] - deliveries_df["purchase_timestamp"]

# 4.
deliveries_df = deliveries_df[deliveries_df["time_diff"].notna()]

# 5.
# time diff as duration in seconds
deliveries_df["time_diff"] = deliveries_df["time_diff"].apply(datetime.timedelta.total_seconds)

# 6.
# deliveries_df = deliveries_df[deliveries_df["time_diff"] >= 0]

# inserting a column with number of RETURNS in particular week

## join deliveries with sessions

In [ ]:
# drop rows where event_type is not equal "BUY_PRODUCT"
sessions_df = sessions_df[sessions_df["event_type"] == "BUY_PRODUCT"]
df = deliveries_df.merge(sessions_df, on="purchase_id", how="left")

In [ ]:
# making sure, that timestamp == purchase_timestamp
num_of_rows_before = df.shape[0]
df = df[df["timestamp"] == df["purchase_timestamp"]]
num_of_rows_after = df.shape[0]

assert(num_of_rows_before == num_of_rows_after)

# now we can drop timestamp column, as it is redundant
df = df.drop(columns="timestamp")

## join with other tables

In [ ]:
df = df.merge(users_df, on="user_id", how="left")
df = df.merge(products_df, on="product_id", how="left")

# missing data analysis - MCAR, MAR, MNAR

made without outliers but with prices below zero (on copy of df)

In [ ]:
missing_data_df = df.copy(deep=False)
missing_data_df["delivery_company_is_missing"] = missing_data_df["delivery_company"].isna()
missing_data_df["user_id_is_missing"] = missing_data_df["user_id"].isna()
missing_data_df["product_id_is_missing"] = missing_data_df["product_id"].isna()

In [ ]:
# rejecting outliers for given PRICE_TRESHOLD
missing_data_df = missing_data_df[missing_data_df["price"] <= PRICE_TRESHOLD]

# rejecting outliers for given WEIGHT_TRESHOLD
missing_data_df = missing_data_df[missing_data_df["weight_kg"] <= WEIGHT_TRESHOLD]

In [ ]:
NUM_BINS_MISSING = 50

def compare_histograms_for_missing(input_df1, input_df2, end_of_title1="", end_of_title2=""):
    fig, ax = plt.subplots(4, 2)
    
    def plot_histograms_missing(input_df, plot_column, end_of_title=""):

        def plot_hist_missing(x, y, col_name, num_bins=None):
            if num_bins:
                ax[x, y].hist(input_df[col_name], bins=num_bins)
            else:
                ax[x, y].hist(input_df[col_name])
            ax[x, y].set_title(f"histogram of {col_name}" + end_of_title)
            ax[x, y].set_xlabel(col_name)
            ax[x, y].set_ylabel("# of observations")

        plot_hist_missing(0, plot_column, "time_diff", NUM_BINS_MISSING)
        plot_hist_missing(1, plot_column, "offered_discount", NUM_BINS_MISSING)
        plot_hist_missing(2, plot_column, "price", NUM_BINS_MISSING)
        plot_hist_missing(3, plot_column, "weight_kg", NUM_BINS_MISSING)

    plot_histograms_missing(input_df1, 0, end_of_title1)
    plot_histograms_missing(input_df2, 1, end_of_title2)

    fig.set_size_inches([24, 21])
    plt.show()

## delivery_company missing

In [ ]:
no_missing_delivery_company = missing_data_df[missing_data_df["delivery_company_is_missing"] == False]
missing_delivery_company = missing_data_df[missing_data_df["delivery_company_is_missing"] == True]

In [ ]:
if MAKE_PLOTS:
    compare_histograms_for_missing(no_missing_delivery_company, missing_delivery_company, " without missing data for delivery_company", " with missing delivery_company")

## user_id missing

In [ ]:
no_missing_user_id = missing_data_df[missing_data_df["user_id_is_missing"] == False]
missing_user_id = missing_data_df[missing_data_df["user_id_is_missing"] == True]

In [ ]:
if MAKE_PLOTS:
    compare_histograms_for_missing(no_missing_user_id, missing_user_id, " without missing data for user_id", " with missing user_id")

## product_id missing
this analysis doesn't make sense

In [ ]:
no_missing_product_id = missing_data_df[missing_data_df["product_id_is_missing"] == False]
missing_product_id = missing_data_df[missing_data_df["product_id_is_missing"] == True]

In [ ]:
# if MAKE_PLOTS:
#     compare_histograms_for_missing(no_missing_product_id, missing_product_id, " without missing data for product_id", " with missing product_id")

# normalize optional_attributes

In [ ]:
optional_attributes_df = pd.json_normalize(df['optional_attributes'])

In [ ]:
display(optional_attributes_df.head())
display(optional_attributes_df.info())
display(optional_attributes_df.describe())

In [ ]:
display(optional_attributes_df.nunique())

To co mogłoby nas zainteresować to width_cm, height_cm oraz depth_cm. Niestety atrybuty te występują tylko w około 10% przypadków (1116 na 11452), przez co nie zostaną one wykorzystane.

In [ ]:
# df[['color', 'width_cm', 'height_cm', 'depth_cm']] = pd.json_normalize(df['optional_attributes'])

# visualizations

## time_diff histogram and log-normal distribution test

In [ ]:
if MAKE_PLOTS:
    fig, ax = plt.subplots(1, 2)

    def plot_hist(x, num_bins=50, func=None):
        if func:
            ax[x].hist(func(df["time_diff"]), bins=num_bins)
            ax[x].set_title(f"with {func} function")
        else:
            ax[x].hist(df["time_diff"], bins=num_bins)
            ax[x].set_title(f"without function")
        ax[x].set_xlabel("time difference [seconds]")
        ax[x].set_ylabel("# of observations")

    plot_hist(0)
    plot_hist(1, func=np.log)
    # plot_hist(1, 0, func=np.log2)
    # plot_hist(1, 1, func=np.log10)

    fig.set_size_inches([12, 6])
    plt.show()

## histograms of continuous variables

In [ ]:
NUM_BINS = 50


def plot_histograms(input_df):
    fig, ax = plt.subplots(2, 2)

    def plot_hist(x, y, col_name, num_bins=None):
        if num_bins:
            ax[x, y].hist(input_df[col_name], bins=num_bins)
        else:
            ax[x, y].hist(input_df[col_name])
        ax[x, y].set_title(f"histogram of {col_name}")
        ax[x, y].set_xlabel(col_name)
        ax[x, y].set_ylabel("# of observations")

    plot_hist(0, 0, "time_diff", NUM_BINS)
    plot_hist(0, 1, "offered_discount", NUM_BINS)
    plot_hist(1, 0, "price", NUM_BINS)
    plot_hist(1, 1, "weight_kg", NUM_BINS)

    fig.set_size_inches([12, 12])
    plt.show()

### with outliers

In [ ]:
if MAKE_PLOTS:
    plot_histograms(df)

### without outliers

In [ ]:
# rejecting outliers for given PRICE_TRESHOLD
df = df[df["price"] <= PRICE_TRESHOLD]

# rejecting outliers for given WEIGHT_TRESHOLD
df = df[df["weight_kg"] <= WEIGHT_TRESHOLD]

In [ ]:
if MAKE_PLOTS:
    plot_histograms(df)

### without prices below 0

In [ ]:
# deleting rows with prices below 0
df = df[df["price"] >= 0]

In [ ]:
if MAKE_PLOTS:
    plot_histograms(df)

### without time_diff below 0

In [ ]:
df_with_time_diff_below_0 = df
df = df[df["time_diff"] >= 0]

In [ ]:
if MAKE_PLOTS:
    plot_histograms(df)

## heatmap

### with time_diff below zero

In [ ]:
def update_list_of_columns():
    banned_list_of_columns = ["purchase_id", "delivery_company", "session_id", "user_id", "product_id"]
    columns_list = [col for col in df.columns.values.tolist() if col not in banned_list_of_columns]
    return columns_list

columns_list = update_list_of_columns()

In [ ]:
if MAKE_PLOTS:
    print(df_with_time_diff_below_0.shape)
    ax = sns.heatmap(df_with_time_diff_below_0[columns_list].corr(), square=True, cmap='RdYlGn')

### without time_diff below zero

#### pearson

In [ ]:
if MAKE_PLOTS:
    print(df.shape)
    ax = sns.heatmap(df[columns_list].corr('pearson'), square=True, cmap='RdYlGn')

#### spearman

In [ ]:
if MAKE_PLOTS:
    print(df.shape)
    ax = sns.heatmap(df[columns_list].corr('spearman'), square=True, cmap='RdYlGn')

#### kendall

In [ ]:
# if MAKE_PLOTS:
#     print(df.shape)
#     ax = sns.heatmap(df[columns_list].corr('kendall'), square=True, cmap='RdYlGn')

## Pairplot

In [ ]:
df.isnull().values.any()

In [ ]:
df_temp = df.drop(columns='optional_attributes')
df_temp.nunique().sort_values()

In [ ]:
if MAKE_PLOTS and MAKE_PAIRPLOT:
    columns_to_drop_for_pairplot = ['purchase_id', 'session_id', 'user_id', 'product_id']
    df_for_pairplot = df.drop(columns=columns_to_drop_for_pairplot)
    g = sns.pairplot(df_for_pairplot, kind='scatter', hue='delivery_company')

Na pierwszy rzut oka atrybut delivery_company nie ma tak dużego znaczenia - spodziewaliśmy się większego zróżnicowania.

In [ ]:
if MAKE_PLOTS and MAKE_PAIRPLOT:
    columns_to_drop_for_pairplot = ['purchase_id', 'session_id', 'user_id', 'product_id']
    df_for_pairplot = df.drop(columns=columns_to_drop_for_pairplot)
    g = sns.pairplot(df_for_pairplot, kind='hist')

## visualizing time series

In [ ]:
plt.hist(df['delivery_timestamp'].dt.hour)

In [ ]:
df['purchase_timestamp'].nunique()

In [ ]:
time_series_df = df.copy()
time_series_df['purchase_date'] = time_series_df['purchase_timestamp'].dt.date
time_series_df = time_series_df.groupby(['purchase_date']).mean()
time_series_df.index = pd.to_datetime(time_series_df.index)
# time_series_df = time_series_df.reset_index()
# time_series_df['purchase_date'].nunique()
print(time_series_df.index.nunique())
display(time_series_df.head())
display(time_series_df.tail())

### 1. trend analysis - moving average

In [ ]:
if MAKE_PLOTS:
    moving_average = time_series_df['time_diff'].rolling(
        window=90,
        center=True,
        min_periods=45
    ).mean()
    
    fig, ax = plt.subplots()
    ax.scatter(time_series_df.index, time_series_df['time_diff'], marker='.', color='0.5')
    moving_average.plot(ax=ax)
    plt.show()

Jak widać wartość średnia z 3 miesięcy w okolicach sierpnia lekko rośnie, a w okolicach marca zaczyna powoli spadać. Natomiast można raczej powiedzieć, że trend jest mniej więcej stały.

### 2. time dependence (seasonality) analysis

#### weekly seasonality with indicators

##### function

In [ ]:
# source: https://www.kaggle.com/code/ryanholbrook/seasonality

def seasonal_plot(X, y, period, freq, ax=None):
    if ax is None:
        _, ax = plt.subplots()
    palette = sns.color_palette("husl", n_colors=X[period].nunique(),)
    ax = sns.lineplot(
        x=freq,
        y=y,
        hue=period,
        data=X,
        ci=False,
        ax=ax,
        palette=palette,
        legend=False,
    )
    ax.set_title(f"Seasonal Plot ({period}/{freq})")
    for line, name in zip(ax.lines, X[period].unique()):
        y_ = line.get_ydata()[-1]
        ax.annotate(
            name,
            xy=(1, y_),
            xytext=(6, 0),
            color=line.get_color(),
            xycoords=ax.get_yaxis_transform(),
            textcoords="offset points",
            size=14,
            va="center",
        )
    return ax

##### weekly seasonality plot

In [ ]:
X = time_series_df.copy()

X["day"] = X.index.dayofweek
X["week"] = X.index.week

X["dayofyear"] = X.index.dayofyear
X["year"] = X.index.year

if MAKE_PLOTS:
    fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(11, 6))
    seasonal_plot(X, y="time_diff", period="week", freq="day", ax=ax0)
    seasonal_plot(X, y="time_diff", period="year", freq="dayofyear", ax=ax1)

Powyżej widzimy, że dzień tygodnia ma znaczenie - na przykład z czwartego na piąty dzień tygodnia prawie zawsze widać znaczny spadek średniego czasu dostawy.

Natomiast na wykresie przedstawiającym okres roczny ciężko znaleźć jakieś zależności.

Wykryta sezonowość na przestrzeni tygodnia zostanie uwzględniona w modelu za pomocą wskaźników (indicators) poprzez one-hot encoding.

Warto dodać, że na powyższym wykresie widać, że średnia wartość time_diff dla kolejnych dni mocno skacze góra-dół. Być może są tam jakieś cykle, które uda się wykryć w kolejnym punkcie.

In [ ]:
df['day_of_week'] = df['purchase_timestamp'].dt.dayofweek

#### periodogram - seasonality with fourier features

##### function

In [ ]:
# source: https://www.kaggle.com/code/ryanholbrook/seasonality

def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("1Y") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        [
            "Annual (1)",
            "Semiannual (2)",
            "Quarterly (4)",
            "Bimonthly (6)",
            "Monthly (12)",
            "Biweekly (26)",
            "Weekly (52)",
            "Semiweekly (104)",
        ],
        rotation=30,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax

##### periodogram plot

In [ ]:
if MAKE_PLOTS:
    plot_periodogram(time_series_df["time_diff"])

Zgodnie z wcześniejszymi spostrzeżeniami na powyższym wykresie widzimy silną zależność tygodniową. Widać także brak silnych zależności w przypadku okresów dłuższych - takich jak miesiąc czy rok.

Nie jest natomiast pewne czy udzielimy to w modelu. Nie mamy pewnego pomysłu jak to zrobić - być może by wymagało to dodatkowego modelu, który przewiduje średni czas dostawy danego dnia. Poza tym zależność tygodniowa prawdopodobnie jest skorelowana z tym, jaki jest dzień tygodnia (poprzedni punkt), co zostanie zamodelowane.

### 3. serial dependence (cycles) analysis

#### function

In [ ]:
# source: https://www.kaggle.com/code/ryanholbrook/time-series-as-features

from statsmodels.graphics.tsaplots import plot_pacf

def lagplot(x, y=None, lag=1, standardize=False, ax=None, **kwargs):
    from matplotlib.offsetbox import AnchoredText
    x_ = x.shift(lag)
    if standardize:
        x_ = (x_ - x_.mean()) / x_.std()
    if y is not None:
        y_ = (y - y.mean()) / y.std() if standardize else y
    else:
        y_ = x
    corr = y_.corr(x_)
    if ax is None:
        fig, ax = plt.subplots()
    scatter_kws = dict(
        alpha=0.75,
        s=3,
    )
    line_kws = dict(color='C3', )
    ax = sns.regplot(x=x_,
                     y=y_,
                     scatter_kws=scatter_kws,
                     line_kws=line_kws,
                     lowess=True,
                     ax=ax,
                     **kwargs)
    at = AnchoredText(
        f"{corr:.2f}",
        prop=dict(size="large"),
        frameon=True,
        loc="upper left",
    )
    at.patch.set_boxstyle("square, pad=0.0")
    ax.add_artist(at)
    ax.set(title=f"Lag {lag}", xlabel=x_.name, ylabel=y_.name)
    return ax


def plot_lags(x, y=None, lags=6, nrows=1, lagplot_kwargs={}, **kwargs):
    import math
    kwargs.setdefault('nrows', nrows)
    kwargs.setdefault('ncols', math.ceil(lags / nrows))
    kwargs.setdefault('figsize', (kwargs['ncols'] * 2, nrows * 2 + 0.5))
    fig, axs = plt.subplots(sharex=True, sharey=True, squeeze=False, **kwargs)
    for ax, k in zip(fig.get_axes(), range(kwargs['nrows'] * kwargs['ncols'])):
        if k + 1 <= lags:
            ax = lagplot(x, y, lag=k + 1, ax=ax, **lagplot_kwargs)
            ax.set_title(f"Lag {k + 1}", fontdict=dict(fontsize=14))
            ax.set(xlabel="", ylabel="")
        else:
            ax.axis('off')
    plt.setp(axs[-1, :], xlabel=x.name)
    plt.setp(axs[:, 0], ylabel=y.name if y is not None else x.name)
    fig.tight_layout(w_pad=0.1, h_pad=0.1)
    return fig

#### lags comparison and autocorrelation

In [ ]:
if MAKE_PLOTS:
    fig = plot_lags(time_series_df["time_diff"], lags=7, nrows=2)

#### partial autocorrelation

In [ ]:
if MAKE_PLOTS:
    fig = plot_pacf(time_series_df["time_diff"], lags=150)

Powyższy wykres w okolicach 120 jest prawdopodobnie niepoprawny. Te korelacje też prawdopodobnie odpuścimy, ponieważ musimy pamiętać o tym, że są to korelacje ze średnim czasem dostawy danego dnia, co musiałoby się pośrednio przenieść na przewidywanie czasu dostawy dla konkretnego zamówienia. Z całej powyższej analizy szeregów czasowych ograniczymy się więc do dodania atrybutu z informacją o dniu tygodnia.

In [ ]:
# lag = 119
# display(time_series_df.time_diff.iloc[5:10])
# display(time_series_df.time_diff.iloc[0+lag:5+lag])

# feature engineering

In [ ]:
cols_to_drop_after_feature_engineering = set()

## interactions

### city and street

In [ ]:
df['city_and_street'] = df['city'] + ' ' + df['street']
display(df[['city_and_street', 'city', 'street']])

## purchase timestamp

In [ ]:

df['purchase_datetime_delta'] = (df['purchase_timestamp'] - df['purchase_timestamp'].min())  / np.timedelta64(1,'D')

## mutual information

In [ ]:
feature_eng_df = df.copy()
feature_eng_df = feature_eng_df.infer_objects()
feature_eng_df = feature_eng_df.set_index('purchase_timestamp').sort_index()
feature_eng_df.info()

In [ ]:
cols_to_drop_feature_eng = ['delivery_timestamp',
                            'event_type',
                            'optional_attributes']
feature_eng_df = feature_eng_df.drop(columns=cols_to_drop_feature_eng)

In [ ]:
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif


def make_mi_scores(in_df, y, reg_version=True):
    X = in_df.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    # display(X.info())
    # display(X.nunique())
    # display(X)
    # display(X.describe())
    if reg_version:
        mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    else:
        mi_scores = mutual_info_classif(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

def create_mi_plot(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

def plot_mi_for_col(in_df, col_name='time_diff', reg_version=True):
    X = in_df.copy()
    y = X.pop(col_name)
    mi_scores = make_mi_scores(X, y, reg_version)
    create_mi_plot(mi_scores)

In [ ]:
mi_score_df = feature_eng_df.copy()

# these columns give errors in mutual_info_regression
# changing their type to float makes them seen as continuous instead of discrete
mi_score_df['purchase_id'] = mi_score_df['purchase_id'].astype(float)
mi_score_df['session_id'] = mi_score_df['session_id'].astype(float)

### for target (time diff)

In [ ]:
if MAKE_PLOTS:
    plot_mi_for_col(mi_score_df, 'time_diff')

- street, name i user_id są prawdopodobnie nadmiarowe i wystarczy wybrać jeden z tych atrybutów. Zostanie to potwierdzone w następnym punkcie.
- city jest istotnym atrybutem, co widać także na poniższych wykresach
- product_name, product_id i weight_kg być może także są redundantne. Zostanie to sprawdzone dalej.
- weight_kg na pewno będzie brane pod uwagę przez korelację liniową
- decyzje co do pozostałych atrybutów zostaną podjęte później

In [ ]:
if MAKE_PLOTS:
    g = sns.scatterplot(x='weight_kg', y='time_diff', data=df)

In [ ]:
if MAKE_PLOTS:
    g = sns.lmplot(x='weight_kg', y='time_diff', col='city', hue='city', col_wrap=4, data=df)

In [ ]:
if MAKE_PLOTS:
    g = sns.catplot(x='delivery_company', y='time_diff', data=df, kind='boxen')

### for street

In [ ]:
if MAKE_PLOTS:
    plot_mi_for_col(mi_score_df, 'street', reg_version=False)

Jak widać dla street atrybuty name, i user_id są reduntantne. Z tych trzech atrybutów wybrany zostanie street, ponieważ w razie przeprowadzki wciąż będzie on niósł ze sobą ważną informację, podczas gdy nazwa użytkownika lub jego identyfikator pozostanie taki sam i może powodować gorsze działanie modelu.

Oprócz tego atrybuty session_id i purchase_id także zostaną od razu dodane do listy atrybutów, które należy wyrzucić - są to tylko identyfikatory, które generowane są dla każdego nowego zamówienia i nie ma w tym żadnych zależności.

In [ ]:
cols_to_drop_after_feature_engineering.update(['name', 'user_id', 'session_id', 'purchase_id'])
mi_score_df = mi_score_df.drop(columns=list(cols_to_drop_after_feature_engineering))

### for product_name

In [ ]:
if MAKE_PLOTS:
    plot_mi_for_col(mi_score_df, 'product_name', reg_version=False)

Tutaj wybrany zostanie product_id, a odrzucony product_name.

In [ ]:
cols_to_drop_after_feature_engineering.update(['product_name'])
mi_score_df = mi_score_df.drop(columns='product_name')

# dropping columns (choosing attributes)

In [ ]:
df.info()

Oprócz kolumn zdefiniowanych do odrzucenia w sekcji feature engineering, odrzucamy także:
1. delivery_timestamp, ponieważ przeciwnym wypadku mielibyśmy przeciek danych (informacja z przyszłości, dzięki której dokładnie moglibyśmy przewidzieć czas dostawy)
2. event_type, ponieważ jest tam tylko jedna wartość

In [ ]:
# drop columns
ADDITIONAL_COLUMNS_TO_DROP = ["delivery_timestamp",
                              "session_id",
                              "purchase_id",
                              "event_type",
                              "name",
                              "city_and_street",
                              "brand",
                              "user_id",
                              'product_name',
                              'offered_discount']
df = df.drop(columns=ADDITIONAL_COLUMNS_TO_DROP)
df = df.drop(columns="optional_attributes") # chyba do zmiany - wysokosc itp.
df = df.drop(columns="purchase_timestamp") # na pewno do zmiany

In [ ]:
df.info()

# one-hot encoding

In [ ]:
# df.to_excel("data_before_one_hot_encoding.xlsx")

In [ ]:
def one_hot_encode_a_col_in_pd(df, col_name):
    one_hot = pd.get_dummies(df[col_name], drop_first=True)
    df = df.drop(columns=col_name)
    df = df.join(one_hot)
    return df

In [ ]:
COLUMNS_TO_ONE_HOT = ["delivery_company", "city", "category_path", "street", 'day_of_week', 'product_id']

for col_name in COLUMNS_TO_ONE_HOT:
    df = one_hot_encode_a_col_in_pd(df, col_name)

#### checking df shape

In [ ]:
print(df.shape)
columns_list = update_list_of_columns()
# ax = sns.heatmap(df[columns_list].corr(), square=True, cmap='RdYlGn')

In [ ]:
df = df.dropna()
print(df.shape)
# one-hot encoding took care of missing data, so shape has not changed

In [ ]:
df.info()

# normalization

In [ ]:
# specify columns for standardization scaling (Z-score normalization)
cols_to_std = []

# specify columns for min-max scaling
# offered_discount, price, weight_kg, purchase_datetime_delta
cols_to_min_max = ['price', 'weight_kg', 'purchase_datetime_delta']
# cols_to_min_max = ['weight_kg']

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

for col in cols_to_std:
    x = df[col].values
    std_scaler = StandardScaler()
    x_scaled = std_scaler.fit_transform(x.reshape(-1, 1))
    df[col] = x_scaled

for col in cols_to_min_max:
    x = df[col].values
    min_max_scaler = MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x.reshape(-1, 1))
    df[col] = x_scaled